In [1]:
import pandas as pd

# Import Dataset

In [3]:
# Import df_CS.csv as df
df = pd.read_csv('df_CS.csv')

# Display the first few rows of the DataFrame
df.head()

,record_id,covid_group,interventiongroup,mmrc_score,pcfs_score,subject_dob,subject_gender,subject_ethnicity,health_smoking,health_history,...,tdi_sum,odi_sum,ndi_sum,anxiety_sum,depression_sum,ptsd_sum,anxiety_dichotomous,depression_dichotomous,ptsd_dichotomous,sfpa_sum
0,1,1.0,1.0,2.0,2.0,1992-04-16,Female,Latino,0.0,hypothyroid,...,0.0,7.0,9.0,3.0,3.0,7.0,1,1,1,24.0
1,2,1.0,0.0,2.0,2.0,1940-03-13,Female,White,0.0,lung cancer 2016,...,0.0,6.0,0.0,3.0,2.0,10.0,1,1,1,23.0
2,3,1.0,1.0,4.0,4.0,1976-11-24,Female,Hispanic,NaN,asthma,...,0.0,17.0,21.0,2.0,4.0,5.0,0,1,1,17.0
3,4,1.0,0.0,1.0,2.0,1986-04-30,Male,White,NaN,"anxiety, depression",...,0.0,4.0,15.0,4.0,4.0,7.0,1,1,1,28.0
4,5,1.0,0.0,3.0,3.0,1963-12-26,Male,White,NaN,NaN,...,0.0,23.0,27.0,3.0,4.0,6.0,1,1,1,18.0


# Outcome Measures

In [4]:
outcome_measures = {

# Autonomic Measures
    'hrv_sdnn': {'name': 'HRV SDNN', 'unit': 'ms'},
    'hrv_rmssd': {'name': 'HRV RMSSD', 'unit': 'ms'},
    'hrv_heartrate': {'name': 'Heart Rate', 'unit': 'bpm'},
    'hrv_lf': {'name': 'HRV LF Power', 'unit': 'ms²'},
    'hrv_hf': {'name': 'HRV HF Power', 'unit': 'ms²'},
    'hrv_lfhf': {'name': 'HRV LF/HF Ratio', 'unit': ''},
    'cpet_hrr1': {'name': 'HRR 1', 'unit': 'BPM'},
    'cpet_hrr2': {'name': 'HRR 2', 'unit': 'BPM'},
    'cpet_chronindex': {'name': 'Chronotropic Index (CI)', 'unit': ''},

}

# Descriptive Statistics

In [5]:
# Print the mean and standard deviation for each outcome measure and group by covid_group
for measure, details in outcome_measures.items():
    mean_value = df.groupby('covid_group')[measure].mean()
    std_value = df.groupby('covid_group')[measure].std()
    print(f"{details['name']} ({details['unit']}):")
    print("Mean:\n", mean_value)
    print("Standard Deviation:\n", std_value)
    print("\n")



HRV SDNN (ms):
Mean:
 covid_group
0.0    57.273333
1.0    50.059286
Name: hrv_sdnn, dtype: float64
Standard Deviation:
 covid_group
0.0    15.222728
1.0    17.019428
Name: hrv_sdnn, dtype: float64


HRV RMSSD (ms):
Mean:
 covid_group
0.0    58.893333
1.0    39.270714
Name: hrv_rmssd, dtype: float64
Standard Deviation:
 covid_group
0.0    27.376525
1.0    18.114309
Name: hrv_rmssd, dtype: float64


Heart Rate (bpm):
Mean:
 covid_group
0.0    60.663333
1.0    67.437857
Name: hrv_heartrate, dtype: float64
Standard Deviation:
 covid_group
0.0    12.311971
1.0     6.493544
Name: hrv_heartrate, dtype: float64


HRV LF Power (ms²):
Mean:
 covid_group
0.0    46.890000
1.0    46.726429
Name: hrv_lf, dtype: float64
Standard Deviation:
 covid_group
0.0    13.576940
1.0    13.920351
Name: hrv_lf, dtype: float64


HRV HF Power (ms²):
Mean:
 covid_group
0.0    53.110000
1.0    53.273571
Name: hrv_hf, dtype: float64
Standard Deviation:
 covid_group
0.0    13.576940
1.0    13.920351
Name: hrv_hf, dtyp